$\newcommand{\K}{\mathbb{K}}$
# Musing on implementing semigroup representation theory and software integration

<img src="pictures/banner.png" align="right" width="55%" style="opacity:0.5;filter:alpha(opacity=40);"/>

Nicolas M. Thiéry<br>
LRI, Université Paris Sud
<br><br><br>

Run this live on binder:
<a href="https://mybinder.org/v2/gh/OpenDreamKit/demo-semigroup-representation-theory/master?filepath=demo.ipynb">
<img src="pictures/qrcode.png" width="25%"> TODO: update
</a>

<br>

<img src="pictures/Flag_of_Europe.svg"   width="10%" align="left">
<img src="pictures/odk-elected-logo.svg" width="6%" align="right">


## Abstract

Extending representation theory from finite groups to finite
semigroups brings interesting challenges, combinatorics, and
applications. Almost a decade ago, I proposed an algorithm to compute
the Cartan Matrix of a semigroup algebra -- a combinatorial invariant
that contains information on how projective modules are built from
simple modules. It boils down to computing with finite semigroups,
characters of groups, and combinatorics. Despite this relative
simplicity, and much to my embarrassment, a full production-grade
implementation is just barely in reach.

In this talk, I'll muse on the long road I took, exploring alongside
many colleagues ways to compose existing software (here in C++, GAP,
Sage) and organize community development aiming at flexibility,
performance, ease of deployment, and sustainability. I will also
briefly report on ongoing joint work with my PhD student Balthazar
Charles to adapt the algorithm to modular representations.

### Balthazar's thesis: modular representations of semigroups

## Main thread of the talk

Balthazar is starting a PhD in <span class="modular">modular</span> representation theory of <span class="semigroup">semigroups</span>

### Mathematical landscape

- Representation theory of groups: a grand old topic; 10k+ papers in a century 
- Representation theory of the <span class=sym>symmetric group $\mathfrak{S}_n$</span>: beautiful combinatorics

- <span class=modular>Modular</span> representation theory of groups: 1k+ papers in the past 50 years

- Representation theory of <span class=semigroup>semigroups</span>: dozens of papers in the last decades

- Motto of <span class=semigroup>semigroup theory</span>: reduce to **combinatorics** + **group theory**

### Strategy

- Implement an algorithm of 2010 in full generality<br>
  <span class=semigroup>Computing the Cartan Invariants Matrix of a semigroup</span>
- Adapt this algorithm to the <span class=modular>modular</modular> case
- Explore the representation theory of the <span class=sym>transformation semigroup $T_n$</span>: beautiful combinatorics?

## The mathematical landscape

### Semigroup theory perspective
- Groups as perfect circles
- The imperfection of semigroups: what happens when you can't always run in circle?
- Irreversible steps, ideals (left and right)
- By losing invertibility, did we loose all hopes for structure?
- J-classes, eggbox

### Why bother?

- Representation theory: also a tool for studying processes<br>
  E.g.: Splitting state spaces of large Discrete Markov chains

- Life is imperfect: many processes have irreversible steps!

**Example:** the Tsetlin library
<img src="tsetlin-library.png" width=30%>
- Beautiful eigenvalues
- Explanation: very irreversible process $\Longrightarrow$ basic representation theory

### There are other angels
<img src="pictures/monoidClasses.svg">

**Motto**: reduce to the angels: group theory, combinatorics, linear algebra

### Representation theory of semigroups, in a nutshell
- Construction of simple modules
- Not semi-simple: Simple ≠ Indecomposable
- Composition factors
- Character theory<br>
  $\Longrightarrow$ tool to recover composition factors


- Among indecomposable: projectives the largest ones
- What are there composition factors -> The Cartan Matrix
  A source of combinatorics

### Computing the Cartan Matrix?

**Algorithm from finite dimensional algebras:**
- Decompose the center of the semisimple quotient (MeatAxe-style)
- Recover idempotents
- Compute projective modules
- Recover composition factors
- Linear algebra on $\K S$, if not $End(\K S)$

**Basic insight (T. 2010):**
- Realize that the Cartan Matrix is the character of $\K S$ as $S$-mod-$S$ bimodule
- Computation by characters!

**Algorithm:**
- Compute semigroup structure, conjugacy class representatives $CC$
- Count two sided fixed points for each pair in $CC$ acting on $S$<br>
  Complexity (Mitchell, T, Charles '20): $O\left(\#CC^2\sum_j \#L_j+\#R_j\right)$ group operations
- Compute characters of simple modules<br>
  Probable bottleneck: computing the radical of $\K L$: $O\left(\sum\#L_i^3\right)$
- Embarassingly parallel

**Implementation:**
- Handles a semigroup of size 31k in one hour (intractable before)
- Aperiodic case only!

## Musing on our computational landscape

### Available building blocks

- Group computations: GAP!
- Semigroup computations: GAP, Semigroups (GAP), KBMag (C), Semigroupe (C), libsemigroups (C++), Sage
- (Modular) representation theory of groups: GAP! (but also GAP3+Specht)
- Combinatorial representation theory: Sage
- Linear algebra: MeatAxe, Linbox, Sage, ...
- Coxeter groups: GAP3+Chevie, GAP, Sage
- Markov chains: Mathematica, ...

Question: How to proceed?

### The Unicorn way: “There can be only one”

That's what happens in the technology sector: a single player takes it all (Amazon, AirBnB, Uber, ...) 

Why not for us?

Let's reimplement everything in ~~C++~~, ~~Magma~~, ~~GAP~~, ~~Sage~~, your favorite system
- Ok for a quick focused result<br>
  Maybe that's what I should have done, actually
- Does not scale, due to time scale and man power:<br>
  our software are the result of decades of hard work and deep expertise
- Promotes competition between systems:<br>
  competition for users, developers, resources<br>
  => beaucoup de dégats

### Balthazar's tool box

In [1]:
import sage_annotations
from mygap import mygap
mygap.LoadPackage("Semigroups");

import sage_semigroups
import sage_combinat_widgets

from   sage_explorer import explore
from sage_explorer.sage_explorer import Settings
Settings.add_property('cardinality', predicate=Groups().Finite().__contains__)
Settings.add_property('conjugacy_classes', predicate=Groups().Finite().__contains__)
Settings.add_property('multiplication_table', predicate=Groups().Finite().__contains__)

%display unicode_art
tensor.symbol = " ⊗ "
%run style/odk.py

#### <span class=semigroup>Semigroup theory</a>

In [2]:
T5 = mygap.FullTransformationSemigroup(5)

In [4]:
T3 = mygap.FullTransformationSemigroup(3)
graph = T3.cayley_graph()
graph.set_latex_options(format="dot2tex")
view(graph)

In [5]:
from francy_widget import FrancyWidget
from networkx import DiGraph
g = DiGraph()
g.add_edges_from([(e[0], e[1]) for e in graph.edges()])
FrancyWidget(g)

ModuleNotFoundError: No module named 'francy_widget'

In [6]:
T5.cardinality()

3125

In [7]:
d_classes = T5.d_classes()
for d_class in d_classes:
    print(d_class)

<Green's D-class: IdentityTransformation>
<Green's D-class: Transformation( [ 1, 2, 3, 4, 1 ] )>
<Green's D-class: Transformation( [ 1, 1, 2, 3, 1 ] )>
<Green's D-class: Transformation( [ 3, 1, 3, 1, 3 ] )>
<Green's D-class: Transformation( [ 1, 1, 1, 1, 1 ] )>


In [8]:
G = d_classes[1].schutzenberger_group()
G

Group([ (1,2,3,4), (1,2) ])

In [8]:
G = d_classes[1].schutzenberger_group()
G

Group([ (1,2,3,4), (1,2) ])

#### <span class=modular>Modular representation of groups</span>

In [9]:
reps = G.irreducible_representations(GF(3))
for rho in reps:
    display([matrix(rho(g).gap()) for g in G.group_generators()])

[ (2), (2) ]

[ (1), (1) ]

⎡ ⎛2 2 0⎞  ⎛2 0 0⎞ ⎤
⎢ ⎜2 1 0⎟  ⎜0 2 2⎟ ⎥
⎣ ⎝0 1 1⎠, ⎝0 0 1⎠ ⎦

⎡ ⎛0 1 2⎞  ⎛2 2 0⎞ ⎤
⎢ ⎜2 2 1⎟  ⎜0 1 0⎟ ⎥
⎣ ⎝2 2 0⎠, ⎝0 0 1⎠ ⎦

In [10]:
all( [ rho(g)*rho(h) == rho(g*h) for g in G for h in G ] )

True

#### Sage-GAP lightweight Math-in-the-Middle interface

##### In action

In [11]:
A = T5.algebra(QQ); A

Algebra of <full transformation monoid of degree 5> over Rational Field

In [12]:
A.an_element() ^ 3

58*B                                    + 74*
    Transformation( [ 4, 5, 1, 2, 3 ] )      

B                                    + 76*1 + 72*
 Transformation( [ 3, 4, 5, 1, 2 ] )             

B                                    + 63*B
 Transformation( [ 2, 3, 4, 5, 1 ] )       Transformation( [ 5, 1, 2, 3, 4 ] )

##### How it works
- Enriched libgap handles with
- Semantic carried over using
- Alignments provided as annotations
```python
    @semantic(mmt="Group", variant="multiplicative")
    class Groups:

        class ParentMethods:

            @semantic(gap="GeneratorsOfGroup", codomain=Family[Self])
            @abstract_method
            def group_generators(self):
                pass
```

#### Exploring

In [13]:
explore(G)

SageExplorer(children=(VBox(children=(ExplorerTitle(children=(MathTitle(value='Exploring: Group([ (1,2,3,4), (…

#### <span class=sym>Combinatorial Representation Theory of $\mathfrak S_n$</sym>

In [12]:
StandardTableaux(10).random_element()

┌───┬───┬───┬────┐
│ 1 │ 3 │ 6 │ 10 │
├───┼───┼───┴────┘
│ 2 │ 5 │
├───┼───┤
│ 4 │ 9 │
├───┼───┘
│ 7 │
├───┤
│ 8 │
└───┘

In [15]:
Sym = SymmetricFunctions(QQ['t']);
s = Sym.s()

In [16]:
s[3,1].coproduct()

1 ⊗ s     + s   ⊗ s    + s   ⊗ s     + s   ⊗ s    + s    ⊗ s   + s    ⊗ s    + 
     ┌┬┬┐    ┌┐    ┌┬┐    ┌┐    ┌┬┬┐    ┌┐    ┌┬┐    ┌┬┐    ┌┐    ┌┬┐    ┌┬┐   
     ├┼┴┘    └┘    ├┼┘    └┘    └┴┴┘    ├┤    └┴┘    └┴┘    ├┤    └┴┘    └┴┘   
     └┘            └┘                   └┘                  └┘                 

s    ⊗ s   + s     ⊗ s   + s     ⊗ 1
 ┌┬┐    ┌┐    ┌┬┬┐    ┌┐    ┌┬┬┐   
 ├┼┘    └┘    └┴┴┘    └┘    ├┼┴┘   
 └┘                         └┘     

In [17]:
@interact
def f(p1 = Partition([2,1])._widget_()):
      return s[p1].coproduct()

Interactive function <function f at 0x7f5f6aa9eea0> with 1 widget
  p1: GridViewWidget(value=[2, 1], children=…

#### <span class=modular>Modular</span> combinatorial representation theory of <span class=sym>$\mathfrak S_n$</a>

In [18]:
list(RibbonTableaux([[5,4,3],[2,1]], [2,1], 3))

⎡   .  .  0  0  0    .  .  1  0  0    .  .  0  0  0 ⎤
⎢   .  0  0  2       .  0  0  0       .  1  0  1    ⎥
⎣   1  0  1      ,   1  0  2      ,   2  0  0       ⎦

In [19]:
Sym.llt(3)

level 3 LLT polynomials over Univariate Polynomial Ring in t over Rational Field

### What it took to get there; lessons learned the hard way

#### Example: Interface to GAP

- **Once upon a time:** no interface from my favorite system (MuPAD)
- **2008:** Sage, with it's text interface to GAP!<br>
  Caveats:
  - too slow for low-granularity computation
  - monolithic adapters: only groups can be used as native groups
- **2012:** Sage's libgap: C-level interface to calling GAP! (Volker Braun et al.)<br>
  Caveat: a patched version of GAP
  - hard to maintain
  - hard to package
  - binary incompatible: $\Longrightarrow$ no kernel modules $\Longrightarrow$ no Semigroups
- **2018-2019:** GAP's libgap (Alex Konovolov, Max Horn, Markus Pfeiffer Erik Bray, Dima Pasechnik, Sebastian Gutsche, ...)<br>
- **2018** OSCAR: fast bidirectional calls between GAP and Julia (Sebastian Gutsche et al.)
- **2015-:** Semantic interface to GAP (T. et al.)<br>
  Prototype; needs maturation and users to expand its scope

#### Example: Interface to Semigroupe / libsemigroups
- **Once upon a time**: Semigroupe by Jean-Éric Pin
- **2010**: Cython bindings (T. et al.)<br>
  Caveats:
  - Semigroupe designed as standalone system. Global variable and single semigroup<br>
  $\Longrightarrow$ Never made its way into SageMath
- **2016-**: libsemigroups ( et al.)
- **2017**: Cython bindings (James Mitchell et T.)<br>
  Caveat: too much work to maintain
- **2018**: cppyy bindings<br>
  Caveat: cppyy not yet mature enough
- **2017-2020**: libsemigroups maturation (James Mitchel with help from Florent Hivert, T.:<br>
  API, build system, packaging, ...
- **2019**: libsemigroups+Semigroups packaged in Sage (Dima Pasechnik)
- **2020**: libsemigroups directly usable in Sage via cppyy!<br>

###  Promoting an ecosystem of interoperable software
- A place where software pieces can emerge, live, compete, and die when they are superseeded or not used anymore, without drowning down years of work
- A place that fosters new ideas, and competition at the granularity of ideas
- A place that fosters collaboration between people and communities

### Best practices

#### (Re)Designing our systems to be good citizens of the ecosystem
- Free Software
- Tests and continuous integration
- Mainstream build and installation system
- Mainstream development workflow
- Packaging (conda, linux packages, ...)
- Reduced coupling with dependencies
- Well defined low level API
- Exposing the semantic

#### Externalize and share solutions to common needs
- E.g. Jupyter as user interface / interactive environment

Shameless plug:
- Session: The Jupyter environment for interactive computational mathematics?<br>
  International Congress for Mathematical Software<br>
  Braunschweig, 13-16 July, 2020<br>
  Call for speakers!

#### Externalize reusable specialized high performance libraries
- Limited scope
- Simple API

#### Interfaces

- Language-level interface:
  - full feature access to the other system via remote function calls and handles
  - performance
- Semantic interface:
  - Automatic conversion of objects
  - Adapter

#### Research Software Engineers

#### Large scale collaboration
- Regular joint workshops
- Joint projects
- COST network?